In [1]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import collections
import qrcode

from docxtpl import DocxTemplate
from docx.shared import Mm
import docxtpl
import requests
import os

# ID PROYECTO
proyecto_id = 35567
# ID FUNCIONES
buscar_reg_nuevos = 6705

In [2]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

# ejecuta un query que esta almacenado en un proyecto de amigocloud (generalmente un update),
# requiere id de proyecto e id de query
# retorna cuantas filas fueron afectadas
def ejecutar_query_por_id(id_project, id_query, tipo_sql):
    # obtiene el query basado en el id_project y el id_query
    get_query = amigocloud.get(f'https://app.amigocloud.com/api/v1/projects/{id_project}/queries/{id_query}')
    # se extrae solo el texto del query
    query = get_query['query']
    # ejecuta el query_sql con metodo post y guarda la respuesta
    respuesta_post = ejecutar_query_sql(id_project, query, tipo_sql)
    # retorna el numero de filas afectadas por el query
    return respuesta_post

# convierte de formato YYYY-mm-dd H:M:S+z a d/m/YYYY
def convertir_formato_fecha(fecha):
    new_formato = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")
    return new_formato

In [3]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

# RUTA ONEDRIVE
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
unidad

'C:\\Documents'

In [5]:
ruta

'OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [6]:
ruta_completa

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [7]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [8]:
def crear_lista_imagenes(id_proyecto, source_amigo_id, doc):
    print('buscando fotos')
    query_fotos = {'query': 'select * from gallery_67001 where source_amigo_id=\'{amigo_id}\''.format(amigo_id=source_amigo_id)}
    print(query_fotos)
    project = 'https://app.amigocloud.com/api/v1/projects/' + str(id_proyecto) + '/sql'
    select_fotos = amigocloud.get(project, query_fotos)
    fotos = select_fotos['data']
    lista_fotos_inline = []
    print('descargando fotos')
    for foto in fotos:
        print('descargando una foto')
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta_completa + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta_completa +'/fotos/' + foto['s3_filename'], width=Mm(60))})
    return lista_fotos_inline

In [9]:
def generar_qr(texto):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(texto)
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black", back_color="white")
    ruta_qr = ruta_completa + '/planos/qr_calidad_semilla_aux.png'
    qr_img.save(ruta_qr)
    return ruta_qr

In [10]:
def set_estado(id, estado):
    query = f'UPDATE dataset_365887 SET estado=\'{estado}\', procesado=True WHERE id={id}'
    ejecutar_query_sql(proyecto_id, query, 'post')
    return None

In [11]:
def generar_reportes(df):
    for i, row in df.iterrows():
        insp = {
            'id':row['id'],
            'fecha': convertir_formato_fecha(row['fecha']),
            'tecnico':row['tecnico'],
            'canhero':row['canhero'],
            'propiedad':row['propiedad'].split('--')[0],
            'lote':row['lote'],
            'variedad':row['variedad'],
            'ciclo':row['ciclo'],
            'edad':row['edad'],
            'area':row['area'],
            'obs':row['obs'],
            'num_mus':40,
            'amigo_id':row['amigo_id'],
            't_yemas':row['total_yemas'],
            't_perf':row['total_perf'],
            'p_perf':row['total_perf'] / row['total_yemas'] * 100,
            't_enrai':row['total_enrai'],
            'p_enrai':row['total_enrai'] / row['total_yemas'] * 100,
            't_brota':row['total_brota'],
            'p_brota':row['total_brota'] / row['total_yemas'] * 100,
            't_muer':row['total_muer'],
            'p_muer':row['total_muer'] / row['total_yemas'] * 100,
            't_inviables':row['total_inviables'],
            'p_inviables':row['porcen_inviables'] * 100,
            't_viables':row['total_viables'],
            'p_viables':row['porcen_viables'] * 100,
            'estado':row['estado']
        }
        insp = collections.namedtuple("insp", insp.keys())(*insp.values()) 
        msj = F'''
--UTEA--
--CALIDAD DE SEMILLA--

Codigo de Inspeccion: {insp.id}
Fecha de 2da Inspeccion: {insp.fecha}
Codigo Canero: {insp.canhero.split(' / ')[0]}
Nombre Canero: {insp.canhero.split(' / ')[1]}
Estado de Insp.: {insp.estado}
Tecnico Responsable: {insp.tecnico}'''
        img_qr = generar_qr(msj)
        
        doc = DocxTemplate(ruta_completa + "/templates/tpl_rpt_calidad_semilla.docx")
        source_amigo_id = insp.amigo_id
        inline_fotos = crear_lista_imagenes(proyecto_id, source_amigo_id, doc)
        inline_firma = docxtpl.InlineImage(doc, image_descriptor=ruta_completa + f'/templates/firmas/{insp.tecnico}.png', width=Mm(60))
        inline_qr = docxtpl.InlineImage(doc, image_descriptor=img_qr, width=Mm(30))
        context = {'insp':insp, 'firma':inline_firma, 'fotos':inline_fotos, 'qr':inline_qr }
        doc.render(context)
        ca = insp.canhero.split(' / ')
        file_name = ca[0] + '_CDSE_' + insp.fecha.replace('/', '-') + '_' + ca[1] + '_' + str(insp.id)
        doc.save(ruta_completa + '/informes/_' + file_name + '.docx')
        set_estado(insp.id, insp.estado)

In [12]:
new_registros = ejecutar_query_por_id(proyecto_id, buscar_reg_nuevos, 'get')
df_new_data = pd.DataFrame(new_registros['data'])
if len(df_new_data) > 0:
    df_validos = df_new_data[df_new_data['propiedad'].notna() & df_new_data['total_yemas'].notna()]
    df_invalidos = df_new_data[df_new_data['propiedad'].isna() | df_new_data['total_yemas'].isna()]
    if len(df_validos) > 0:
        generar_reportes(df_validos)
    else:
        print('No existen registros validos nuevos')
    if len(df_invalidos) > 0:
        print('Existen registros con datos no validos')
else:
    print('No existen registros nuevos')

buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='501b540de48c4df5a565354dda668d96'"}
descargando fotos
descargando una foto
descargando una foto
buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='1b1c800812124b329cc6f08df9a9d683'"}
descargando fotos
descargando una foto
buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='338dbbcecde844af86d1022737fd019a'"}
descargando fotos
descargando una foto
descargando una foto
buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='de17e82dfb0246d6be36b3b0063e8e1b'"}
descargando fotos
descargando una foto
buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='6e02085966134712a4254eddfe2660e2'"}
descargando fotos
descargando una foto
descargando una foto
Existen registros con datos no validos
